In [1]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


plt.rcParams["figure.figsize"] = (10,8)

import warnings
warnings.filterwarnings('ignore')
from datetime import datetime 

In [2]:
df = pd.read_csv("datos/bikes.csv", index_col=0)
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [3]:
df.sample(10)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
198,199,18-07-2018,autumn,0,7,0,1,1,1,30.613347,35.19625,65.1250,14.458868,841,3617,4458
362,363,29-12-2018,spring,0,12,0,4,1,1,10.181653,13.19460,57.4167,8.000604,254,2169,2423
567,568,22-07-2019,autumn,1,7,0,0,0,2,27.367500,31.15625,76.2500,6.292694,2544,4866,7410
703,704,05-12-2019,winter,1,12,0,3,1,1,17.971653,21.40060,48.5000,21.709407,331,5398,5729
374,375,10-01-2019,spring,1,1,0,2,1,1,12.656536,15.94130,64.6522,12.565984,173,3425,3598
244,245,02-09-2018,autumn,0,9,0,5,1,2,26.376653,30.24040,72.7083,9.375243,875,3852,4727
295,296,23-10-2018,winter,0,10,0,0,0,1,17.288347,21.11665,74.1250,6.667338,1619,2762,4381
636,637,29-09-2019,winter,1,9,0,6,0,1,22.242500,26.51480,54.2917,15.249468,2589,5966,8555
506,507,22-05-2019,summer,1,5,0,2,1,2,25.215000,29.04125,76.5833,7.917189,819,5254,6073
437,438,14-03-2019,spring,1,3,0,3,1,1,23.472500,27.43085,50.7083,7.709154,997,5315,6312


In [4]:
#df_spring = df[df['season'] == 'spring']

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 97.0+ KB


In [6]:
df['dteday'] = pd.to_datetime(df['dteday'], format= '%d-%m-%Y')

In [7]:
df.sample(20)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
479,480,2019-04-25,summer,1,4,0,3,1,1,19.543347,23.51585,42.7917,7.959064,655,5541,6196
716,717,2019-12-18,winter,1,12,0,2,1,1,16.844153,20.48540,66.6250,14.834068,433,5124,5557
86,87,2018-03-28,summer,0,3,0,1,1,1,10.838268,12.87870,30.2174,14.217668,222,1806,2028
472,473,2019-04-18,summer,1,4,0,3,1,2,18.996653,22.85190,56.9167,11.250104,347,4020,4367
714,715,2019-12-16,winter,1,12,0,0,0,2,14.862500,18.49690,83.8750,6.749714,538,3248,3786
517,518,2019-06-02,summer,1,6,0,6,0,1,23.916653,28.31440,54.9167,12.499654,2795,5325,8120
531,532,2019-06-16,summer,1,6,0,6,0,1,25.898347,29.73540,50.4167,11.166689,2963,4739,7702
529,530,2019-06-14,summer,1,6,0,4,1,1,26.581653,31.21915,56.9583,17.000111,1180,6183,7363
513,514,2019-05-29,summer,1,5,0,2,1,1,29.622500,33.64960,68.4583,19.791800,880,4863,5743
25,26,2018-01-26,spring,0,1,0,3,1,3,8.917500,10.18000,86.2500,19.687950,34,472,506


In [8]:
pd.date_range(start= '2018-03-21', end='2018-06-21')

DatetimeIndex(['2018-03-21', '2018-03-22', '2018-03-23', '2018-03-24',
               '2018-03-25', '2018-03-26', '2018-03-27', '2018-03-28',
               '2018-03-29', '2018-03-30', '2018-03-31', '2018-04-01',
               '2018-04-02', '2018-04-03', '2018-04-04', '2018-04-05',
               '2018-04-06', '2018-04-07', '2018-04-08', '2018-04-09',
               '2018-04-10', '2018-04-11', '2018-04-12', '2018-04-13',
               '2018-04-14', '2018-04-15', '2018-04-16', '2018-04-17',
               '2018-04-18', '2018-04-19', '2018-04-20', '2018-04-21',
               '2018-04-22', '2018-04-23', '2018-04-24', '2018-04-25',
               '2018-04-26', '2018-04-27', '2018-04-28', '2018-04-29',
               '2018-04-30', '2018-05-01', '2018-05-02', '2018-05-03',
               '2018-05-04', '2018-05-05', '2018-05-06', '2018-05-07',
               '2018-05-08', '2018-05-09', '2018-05-10', '2018-05-11',
               '2018-05-12', '2018-05-13', '2018-05-14', '2018-05-15',
      

In [9]:
'2018-05-02' in pd.date_range(start= '2018-03-21', end='2018-06-21')

True

In [10]:
def estaciones(dato):

    primavera = np.union1d(pd.date_range(start= '2018-03-21', end='2018-06-21'),pd.date_range(start= '2019-03-21', end='2019-06-21'))
    verano = np.union1d(pd.date_range(start= '2018-06-22', end='2018-09-22'),pd.date_range(start= '2019-06-22', end='2019-09-22'))
    otoño = np.union1d(pd.date_range(start= '2018-09-23', end='2018-12-21'),pd.date_range(start= '2019-09-23', end='2019-12-21'))
    invierno = np.union1d(pd.date_range(start= '2018-12-22', end='2018-03-20'),pd.date_range(start= '2019-12-22', end='2019-03-20'))

    if dato in primavera:
        return 'spring'
    elif dato in verano:
        return 'summer'
    elif dato in otoño:
        return 'autumn'
    else:
        return 'winter'

In [11]:
df['season'] = df['dteday'].apply(estaciones)

In [12]:
df.sample(20)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
240,241,2018-08-29,summer,0,8,0,1,1,1,26.103347,30.39790,55.4583,10.708275,729,3905,4634
88,89,2018-03-30,spring,0,3,0,3,1,2,12.300000,14.86750,64.6667,11.583496,168,1368,1536
697,698,2019-11-29,autumn,1,11,0,4,1,1,11.515670,14.92110,55.5652,7.739974,243,5080,5323
29,30,2018-01-30,winter,0,1,0,0,0,1,8.877402,12.51610,72.2174,4.956834,140,956,1096
118,119,2018-04-29,spring,0,4,0,5,1,1,20.910000,24.87315,45.7083,16.084221,878,3717,4595
450,451,2019-03-27,spring,1,3,0,2,1,1,13.256653,15.78270,29.0000,12.541864,531,4571,5102
219,220,2018-08-08,summer,0,8,0,1,1,1,31.365000,35.16415,59.2083,12.875725,846,3480,4326
359,360,2018-12-26,winter,0,12,1,1,0,1,13.191299,15.77675,50.6957,16.044155,430,887,1317
303,304,2018-10-31,autumn,0,10,0,1,1,1,13.940000,17.80315,70.3333,7.125450,362,3307,3669
207,208,2018-07-27,summer,0,7,0,3,1,1,31.775000,34.53335,40.2917,12.292021,755,3901,4656
